# Parse Tree Readability

This Jupyter notebook is for developing, explaining, and experimenting with parse tree readability, a measure of readability I'm developing based on the premises that word length and sentence length are not the most important qualities of a piece of text when it comes to determining the difficulty of understanding it.

## Libraries and Imports

We'll need several libraries for the corpora, for simple text analysis, and for parsing sentences.

In addition, we'll set up this notebook's graphing displays and create global variables for global things, such as American English, `en_nlp`; and a grammar derived from the Penn Treebank, which, when we need it, will be initialized.

In [ ]:
%matplotlib inline
import math
from math import sqrt, log
from collections import Counter
import nltk
from nltk import Tree, Production, Nonterminal, CFG, ChartParser
import textacy
import pyphen  # hyphenation library
import spacy

pyphen.language_fallback('en_US')
dic = pyphen.Pyphen(lang='en_US')

en_nlp = spacy.load('en')
productions = None  # to be created as needed, later

The first thing we'll do is pull in our corpus. The corpus needs to contain a variety of reading levels within it.

Additionally, we need to have an already-parse corpus. We can use the Penn Treebank for this.

## Dealing with Parse Tree

Sometimes we'll need to get back the original sentence from a parse tree.

In [ ]:
def recreate_sentence(parse_tree):
    """
    Get back the original text of a sentence.
    """
    leading_tags = set(['(', '$', '``'])  # don't need a space after, but do before
    following_tags = set([')', ',', '.', ':', "''", 'POS'])  # need a space after, but not before
    sentence = ''
    no_sp = False
    for w, pos in parse_tree.pos():
        if pos != '-NONE-':
            if pos in following_tags or w in {'%'}:
                sentence += w
            else:
                sentence += ('' if no_sp else ' ') + w
            no_sp = pos in leading_tags
    return sentence[1:]  # ignore leading space

def recreate_sentences(parse_trees):
    return [recreate_sentence(parse_tree) for parse_tree in parse_trees]

In [ ]:
wsj = nltk.corpus.treebank.parsed_sents('wsj_0012.mrg')
w = wsj[0]
recreate_sentences(wsj)

In [ ]:
production = w.productions()[0]
print(production)

If I'm able to, I'd like to use tagged, but non-parsed sentences.

TODO: get some small texts which are tagged with Penn Treebank tags


So, what are the reading levels of these per the current gold standard SMOG?

In [ ]:
# TODO: get the SMOG scores of each one

Recap of the SMOG scores.

[[ TODO: if I'm going to not use individual works, but per-genre works, I'll need to be politic about leaping to conclusions. ]]

Next, let's begin to create our parsing functions. These are going to start simply, and get more complicated.

## Textual metrics

Let's begin with text-based metrics: sentence length and syllabification. We'll use the syllabification library `pyphen` to count the syllables in each word.

We can build from these to create the standard text-based metrics, such as SMOG, Flesch-Kincaid, etc.

Also note that every one of these takes a POS-tagged sentence, such as in the Treebank corpus, arranged as a list of (word/punctuation, POS) tuples. They do not take text, so it's important, if you want to use these on text, to tag the text beforehand. This can be done with, e.g., spaCy:

```python
doc = en_nlp(u'One morning, when Gregor Samsa woke from troubled dreams, \
he found himself transformed in his bed into a horrible vermin. He lay on \
his armour-like back, and if he lifted his head a little he could see his \
brown belly, slightly domed and divided by arches into stiff sections.')
[[(word.text, word.tag_) for word in sent] for sent in doc.sents]
# returns [[('One', 'CD'), ('morning', 'NN'), (',', ','), ('when', 'WRB'), ...], ...]
```

In [ ]:
def depunctuate(tagged_sentence):
    """
    From a tagged sentence (as a list), returns a sentence without punctuation
    """
    punct_tags = set(['(', ')', ',', '--', '.', ':', '-NONE-', '``', "''", '$'])
    return ((word, tag) for word, tag in tagged_sentence
            if tag not in punct_tags)
    
def n_words(tagged_sentence):
    """
    From a tagged sentence (as a list), returns the number of words in it,
    sans punctuation.
    """
    return sum(1 for word in depunctuate(tagged_sentence))

def word_lengths(tagged_sentence):
    """
    From a tagged sentence (as a list), returns a list of non-punctation
    word lengths
    """
    return [len(word) for word, pair in depunctuate(tagged_sentence)]

def avg_word_length(tagged_sentence):
    """The average word length of the sentence"""
    lengths = word_lengths(tagged_sentence)
    return sum(lengths) / len(lengths)

def syllables(tagged_sentence):
    """
    From a tagged sentence (as a list), returns a list of non-punctation
    syllables per word.
    """
    return (len(dic.positions(word)) + 1
            for word, pair in depunctuate(tagged_sentence))

def n_monosyllable_words(tagged_sentence):
    """
    Returns the number of one syllable words in the (word, tag)-list sentence
    """
    return len([sylls for sylls in syllables(tagged_sentence) if sylls == 1])

def n_polysyllable_words(tagged_sentence):
    """
    Returns the number of 3+ syllable words in the (word, tag)-list sentence
    """
    return len([sylls for sylls in syllables(tagged_sentence) if sylls >= 3])

Testing these functions on a few sentences

In [ ]:
w0, w1, w2 = wsj[:3]
print(w1.leaves(), '\n')
print(list(syllables(w1.pos())))
print(n_polysyllable_words(w0.pos()))
print(w1)

## Standard Readability Metrics

With these building blocks in place, we can create our own versions of the standard readability metrics.

In [ ]:
def SMOG(text):
    """
    Computes the SMOG score of a piece of text, in this case, a list of tagged sentences.
    There should be at least 30 sentences in the text.
    
    McLaughlin, G. Harry (May 1969). "SMOG Grading — a New Readability Formula" (PDF).
    Journal of Reading. 12 (8): 639–646.
    """
    n_polysyllables = sum(n_polysyllable_words(sentence) for sentence in text)
    n_sentences = len(text)
    grade = 1.0430 * sqrt(n_polysyllables * (30 / n_sentences)) + 3.1291
    return grade
    
def flesch_kincaid_grade_level(text):
    """
    Computes the Flesch-Kincaid grade level of a piece of text, in this case,
    a list of tagged sentences.

    Kincaid JP, Fishburne RP Jr, Rogers RL, Chissom BS (February 1975).
    "Derivation of new readability formulas (Automated Readability Index,
    Fog Count and Flesch Reading Ease Formula) for Navy enlisted personnel".
    Research Branch Report 8-75, Millington, TN: Naval Technical Training,
    U. S. Naval Air Station, Memphis, TN.
    """
    n_sentences = len(text)
    n_words_ = sum(n_words(sentence) for sentence in text)
    n_syllables = sum(sum(syllables(sentence)) for sentence in text)
    grade = 0.39 * (n_words_ / n_sentences) + 11.8 * (n_syllables/n_words_) - 15.59
    return grade

def dale_chall(text):
    """
    Computes the Dale-Chall Formula for readability, 1961 revision, for a list
    of tagged sentences
    
    McCallum, D. and Peterson, J. (1982), Computer-Based Readability Indexes,
    Proceedings of the ACM '82 Conference, (October 1982), 44-48. 
    """
    f = open('dale-chall.txt')
    dale_long_list = [word.lower() for word in f.read().split()]
    f.close()
    
    n_dale_long = sum(1 for sentence in text for word, POS in sentence
                      if word.lower() in dale_long_list)
    n_words_ = sum(n_words(sentence) for sentence in text)
    n_sentences = len(text)
    grade = 14.863 - 11.42 * (n_dale_long / n_words_) + 0.0512 * (n_words_ / n_sentences)
    return grade

In [ ]:
class TextStats:

    def __init__(self, corpus):
        # The corpus is a tagged sentence or list of tagged sentences
        self.corpus = [corpus] if isinstance(corpus[0], tuple) else corpus

    def basic_counts(self):
        n_s = len(self.corpus)
        return {
            'n_sentences': n_s,
            'n_words': sum(n_words(s) for s in self.corpus),
            'n_chars': sum(sum(word_lengths(s)) for s in self.corpus),
            'syllables': sum(sum(syllables(s)) for s in self.corpus),
            'n_monosyllable_words': sum(n_monosyllable_words(s)
                                        for s in self.corpus),
            'n_polysyllable_words': sum(n_polysyllable_words(s)
                                        for s in self.corpus),
        }

    def get_stats(self):
        return {
            'SMOG': SMOG(self.corpus),
            'flesch_kincaid_grade_level': flesch_kincaid_grade_level(self.corpus),
            'dale_chall': dale_chall(self.corpus),
        }

    @staticmethod
    def depunctuate(tagged_sentence):
        """
        From a tagged sentence (as a list), returns a sentence without punctuation
        """
        punct_tags = set(['(', ')', ',', '--', '.', ':',
                          '-NONE-', '``', "''", '$'])
        return ((word, tag) for word, tag in tagged_sentence
                if tag not in punct_tags)

    @staticmethod
    def n_words(tagged_sentence):
        """
        From a tagged sentence (as a list), returns the number of words in it,
        sans punctuation.
        """
        return sum(1 for word in depunctuate(tagged_sentence))

    @staticmethod
    def word_lengths(tagged_sentence):
        """
        From a tagged sentence (as a list), returns a list of non-punctation
        word lengths
        """
        return [len(word) for word, pair in depunctuate(tagged_sentence)]

    @staticmethod
    def avg_word_length(tagged_sentence):
        """The average word length of the sentence"""
        lengths = word_lengths(tagged_sentence)
        return sum(lengths) / len(lengths)

    @staticmethod
    def syllables(tagged_sentence):
        """
        From a tagged sentence (as a list), returns a list of non-punctation
        syllables per word.
        """
        return (len(dic.positions(word)) + 1
                for word, pair in depunctuate(tagged_sentence))

    @staticmethod
    def n_monosyllable_words(tagged_sentence):
        """
        Returns the number of one syllable words in the (word, tag)-list sentence
        """
        return len([sylls for sylls in syllables(tagged_sentence) if sylls == 1])

    @staticmethod
    def n_polysyllable_words(tagged_sentence):
        """
        Returns the number of 3+ syllable words in the (word, tag)-list sentence
        """
        return len([sylls for sylls in syllables(tagged_sentence) if sylls >= 3])

    @staticmethod
    def SMOG(text):
        """
        Computes the SMOG score of a piece of text, in this case, a list of
        tagged sentences. There should be at least 30 sentences in the text.

        McLaughlin, G. Harry (May 1969). "SMOG Grading — a New Readability
        Formula" (PDF). Journal of Reading. 12 (8): 639–646.
        """
        n_polysyllables = sum(n_polysyllable_words(sentence)
                              for sentence in text)
        n_sentences = len(text)
        grade = 1.0430 * sqrt(n_polysyllables * (30 / n_sentences)) + 3.1291
        return grade

    @staticmethod
    def flesch_kincaid_grade_level(text):
        """
        Computes the Flesch-Kincaid grade level of a piece of text, in this case,
        a list of tagged sentences.

        Kincaid JP, Fishburne RP Jr, Rogers RL, Chissom BS (February 1975).
        "Derivation of new readability formulas (Automated Readability Index,
        Fog Count and Flesch Reading Ease Formula) for Navy enlisted personnel".
        Research Branch Report 8-75, Millington, TN: Naval Technical Training,
        U. S. Naval Air Station, Memphis, TN.
        """
        n_sentences = len(text)
        n_words_ = sum(n_words(sentence) for sentence in text)
        n_syllables = sum(sum(syllables(sentence)) for sentence in text)
        grade = 0.39 * (n_words_ / n_sentences) + 11.8 * \
            (n_syllables/n_words_) - 15.59
        return grade

    @staticmethod
    def dale_chall(text):
        """
        Computes the Dale-Chall Formula for readability, 1961 revision, for a
        list of tagged sentences.

        McCallum, D. and Peterson, J. (1982), Computer-Based Readability
        Indexes, Proceedings of the ACM '82 Conference, (October 1982), 44-48.
        """
        f = open('dale-chall.txt')
        dale_long_list = [word.lower() for word in f.read().split()]
        f.close()

        n_dale_long = sum(1 for sentence in text for word, POS in sentence
                          if word.lower() in dale_long_list)
        n_words_ = sum(n_words(sentence) for sentence in text)
        n_sentences = len(text)
        grade = 14.863 - 11.42 * (n_dale_long / n_words_) + \
            0.0512 * (n_words_ / n_sentences)
        return grade

In [ ]:
wsj_pos = [s.pos() for s in wsj]
TextStats(wsj_pos).get_stats()
TextStats(wsj_pos).basic_counts()

In [ ]:
# Compare this to the output of textacy -- it's close!
wsj_pos = [s.pos() for s in wsj]
text = '\n'.join(recreate_sentences(wsj))
doc = textacy.Doc(text)

# t_words = list(textacy.text_stats.extract.words(doc, filter_punct=True, filter_stops=False, filter_nums=False))
# my_words = (word for sentence in wsj_pos for word, pos in depunctuate(sentence))
# print(list(zip(t_words, my_words)))
# print('*' * 80)

ts = textacy.text_stats.TextStats(doc)
print(ts.basic_counts)
print('n_sents:', len(wsj_pos))
print('n_words:', sum(n_words(sentence) for sentence in wsj_pos))
print('n_syllables', sum(sum(syllables(sentence)) for sentence in wsj_pos))
print('n_monosyllable_words', sum(n_monosyllable_words(sentence) for sentence in wsj_pos))
print('n_polysyllable_words', sum(n_polysyllable_words(sentence) for sentence in wsj_pos))

print('*' * 80)
print('SMOG:', SMOG(wsj_pos))
print('FK:', flesch_kincaid_grade_level(wsj_pos))
print('Dale-Chall:', dale_chall(wsj_pos))
print(ts.readability_stats)

For the next round of text analysis functions, let's dig a little deeper into the POS analysis. Remember that this is still sans parse-tree-ification.

## Tag-based Metrics

These use the POS tags. Since we're using the Penn Treebank, we need to make sure that whatever tags we're using are within this tagset; some tagged texts from the Brown Corpus, for example, use other tagsets.

In [ ]:
def tag_sentence(sentence):
    """A helper method to easily tokenize a sentence"""
    return nltk.pos_tag(nltk.word_tokenize(sentence))


def tag_sentences(sentences):
    """A helper method to easily tokenize sentences"""
    return [tag_sentence(sent) for sent in nltk.sent_tokenize(sentences)]


class TagStats:

    def __init__(self, corpus):
        def n_repeated_possessives(tagged_sentence):
            """
            The number of 2+ repeated possessives, e.g.:
            "John's mother's neighbor's uncle's dog's Instagram account" -> 5
            "John's dog's Instagram account" -> 2
            "John's Instagram account" -> 0  # because it's not repeated
            """
            max_count = 0

            count = 0
            loc = -1
            for i, (word, pos) in enumerate(tagged_sentence[::2]):
                if pos == 'POS':
                    count += 1
                    max_count = max(max_count, count)
                    if loc != i - 1:  # it's not repeated
                        count = 0
                    loc = i

            count = 0
            loc = -1
            for i, (word, pos) in enumerate(tagged_sentence[1::2]):
                if pos == 'POS':
                    count += 1
                    max_count = max(max_count, count)
                    if loc != i - 1:  # it's not repeated
                        count = 0
                    loc = i

            return max_count + 1 if max_count >= 1 else 0

        def n_repeated_adverbs(tagged_sentence):
            """
            The number of 2+ repeated adverbs (RB, RBR, and RBS/RBT) e.g.:
            "harder better faster stronger" -> 4  # as adverbs, not adjectives!
            "most happily" -> 2
            "likely ready" -> 0  # because "ready" is an adjective
            """
            adverbs = {'RB', 'RBR', 'RBS', 'RBT', 'RB$',
                       'RB+BEZ', 'RB+CS', 'RBR+CS', 'RN', 'RP'}
            max_count = 0

            count = 0
            loc = -1
            for i, (word, pos) in enumerate(tagged_sentence):
                if pos in adverbs:
                    count += 1
                    max_count = max(max_count, count)
                    if loc != i - 1:  # it's not repeated
                        count = 0
                    loc = i

            return max_count if max_count >= 2 else 0

        # the corpus is a tagged sentence or list of tagged sentences
        self.corpus = [corpus] if isinstance(corpus[0], tuple) else corpus

        self.POSs = [set(pos for word, pos in sentence) for sentence in corpus]
        self.n_pronouns = [len([pos for word, pos in sentence if pos == 'PRP'])
                           for sentence in corpus]
        self.n_repeated_possessives = [n_repeated_possessives(sentence)
                                       for sentence in corpus]
        self.n_repeated_adverbs = [n_repeated_adverbs(sentence)
                                   for sentence in corpus]

    def get_stats(self):
        """Combine all the statistics together"""
        ...
        n_s = len(self.corpus)
        t_pos = len({POS for POSs in self.POSs for POS in POSs})
        a_pos = sum(len(POSs) for POSs in self.POSs) / n_s
        a_r_p = sum(x for x in self.n_repeated_possessives) / n_s
        a_r_a = sum(x for x in self.n_repeated_adverbs) / n_s

        return {
            'total_POSs': t_pos,
            'avg_POSs': a_pos,
            'avg_pronouns': a_pro,
            'avg_repeated_possessives': a_r_p,
            'avg_repeated_adverbs': a_r_a,
        }

In [ ]:
pt = nltk.PerceptronTagger()
model = pt.__dict__['model']
w = model.__dict__['weights']
f = open('tags.txt', 'w')
for x in w.items():
    print(x, file=f)
f.close()

In [ ]:
w = wsj[7].pos()
print(w)
print('POS types:', n_POSs(w))
print('n_pronouns:', n_pronouns(w))
r_p_sent = [('My', 'PRP$'), ('',''), ('dog', 'NN'), ("'s", 'POS'), ('therapist', 'NN'), ("'s", 'POS'),
            ('uncle', 'NN'), ("'s", 'POS'), ('friend', 'NN'), ("'s", 'VBZ'), ('smiling', 'VBG')]

r_a_sent = [('My', 'PRP$'), ('very', 'RB'), ('very', 'RB'), ('very', 'RB'), ('very', 'RB'), ('good', 'JJ'),
            ('dog', 'NN')]

print('n_rep_poss:', n_repeated_possessives(r_p_sent))  # should be 3, not 4
print('n_rep_adv:', n_repeated_adverbs(r_a_sent))

## Tree-based Metrics

These use the structure of the parse trees, not the productions (yet!) to measure things. We'll start with simple measurements, and build on top of those.

First, of course, we need to get the parse trees. [ TODO ]

In [ ]:
def init_productions():
    """
    We'll amortize the cost of initializing the grammar, done by reading
    through the Penn Treebank, by only doing it once.
    Note that this creates only non-terminal productions. All of the
    terminal productions need to be created as well, then appended to the
    tb_grammar before creating a parser. Use create_terminals for this.
    """
    global productions
    global branch_factor  # the average length of the rhs
    
    def filter_prod(production):
        """Removes dashed subtypes in nonterminal productions"""
        def filter_subtypes(nonterminal):
            if nonterminal.symbol() == '-NONE-':
                return nonterminal
            else:
                return Nonterminal(nonterminal.symbol().split('-')[0])

        l = filter_subtypes(production.lhs())
        r = tuple(filter_subtypes(p) for p in production.rhs())
        return Production(l, r)

    if productions is None:
        treebank = nltk.corpus.treebank.parsed_sents()
        tb_productions = [production for t in treebank for production in t.productions()
                          if isinstance(production.rhs()[0], Nonterminal)]
        # simplify the productions
        simplified_productions = [filter_prod(p) for p in tb_productions]
        counter = Counter(simplified_productions)
        # 7,982 nonterminal rules -- we need to reduce this greatly
        top_prods = {p for p, c in counter.most_common(250)}  # magic number
        productions = top_prods
        branch_factor = sum(len(p.rhs()) for p in productions) / len(productions)

def create_terminals(tagged_text):
    """
    We need to add terminals to our grammar as well. This takes a tagged
    sentence, e.g. [('The', 'DT'), ...] and returns a list of terminal
    productions.
    The tagged text may also be a list of tagged sentences, in which case
    we add all words from these sentences to terminal productions
    """
    if isinstance(tagged_text[0], tuple):
        return set(Production(Nonterminal(tag), (word,))
                   for word, tag in tagged_text)
    else:
        return set(Production(Nonterminal(tag), (word,))
                   for tagged_sentence in tagged_text
                   for word, tag in tagged_sentence)

def create_parser(tagged_text):
    """
    Return a parser for the tagged text, which, as in `create_terminals`,
    can be either a POS-tagged sentence or a list of them.
    """
    global productions
    init_productions()
    # add tagged words to the parser
    productions |= create_terminals(tagged_text)
    grammar = CFG(start=Nonterminal('S'), productions=productions)
    return nltk.ChartParser(grammar)

def possible_parses(tagged_sentence):
    """
    Similar to the above, but sees how many of these possible productions
    were actually considered by the parser.
    """
    parser = create_parser(tagged_sentence)
    chart = parser.chart_parse((word for word, pos in sent))
    return len(chart.edges())

def get_trees(tagged_sentence):
    """
    Returns all possible parse trees from a sentence.
    The sentence is POS-tagged, and this returns a list of Trees.
    """
    parser = create_parser(tagged_sentence)
    parsing = parser.parse(word for word, pos in tagged_sentence)
    return [tree for tree in parsing]

def n_trees(trees):
    if trees:
        return len(trees)
    else:
        return 1
    
def tree_depth(tree):
    """Returns the tree depth of an individual parse tree. O(n)"""
    return tree.height()

def max_tree_depth(trees, corpus=[]):
    """
    Returns the max tree depth over a collection of parse trees.
    If there are no trees (if there are no parses, e.g.), an optional
    argument of the original corpus is used to provide an estimate.
    """
    if trees:
        return max((tree_depth(tree) for tree in trees))
    elif corpus:
        # Return the average tree height based on branching on the rhs of 
        # productions. I.e., if we have the rule X -> Y Z W, there are three branches,
        # and so on. Averaging over all the production rules will give us a
        # branching factor of, say, 2.6. The average tree height is then 
        # log_bf(tree length) + 5. (5 is a fudge factor)
        ests = (math.ceil(log(len(sentence)/log(branch_factor)) + 5)
               for sentence in corpus)
        return max(ests)
    else:
        return 0

def avg_tree_depth(trees, corpus=[]):
    """
    Returns the max tree depth over a collection of parse trees.
    If there are no trees (if there are no parses, e.g.), an optional
    argument of the original corpus is used to provide an estimate.
    """
    if trees:
        return sum((tree_depth(tree) for tree in trees))/ len(trees)
    elif corpus:
        # Return the average tree height based on branching on the rhs of 
        # productions. I.e., if we have the rule X -> Y Z W, there are three branches,
        # and so on. Averaging over all the production rules will give us a
        # branching factor of, say, 2.6. The average tree height is then 
        # log_bf(tree length) + 2. (2 is the POS and terminal string)
        ests = (log(len(sentence)/log(branch_factor)) + 5
                for sentence in corpus)
        return sum(ests) / len(corpus)
    else:
        return 0
    
def combined(corpus):
    """Avoids extra work. Use this for faster computation."""
    trees = get_trees(corpus)
    m_t_d = max_tree_depth(trees, corpus=corpus)
    a_t_d = avg_tree_depth(trees, corpus=corpus)
    return {
        'n_trees': n_trees(trees),
        'max_tree_depth': m_t_d,
        'avg_tree_depth': a_t_d
    }

In [ ]:
class TreeStats:
    def __init__(self, corpus):
        """
        We'll amortize the cost of initializing the grammar, done by reading
        through the Penn Treebank, by only doing it once.
        Note that this creates only non-terminal productions. All of the
        terminal productions need to be created as well, then appended to the
        tb_grammar before creating a parser. Use create_terminals for this.
        """

        def filter_prod(production):
            """Removes dashed subtypes in nonterminal productions"""
            def filter_subtypes(nonterminal):
                if nonterminal.symbol() == '-NONE-':
                    return nonterminal
                else:
                    return Nonterminal(nonterminal.symbol().split('-')[0])

            l = filter_subtypes(production.lhs())
            r = tuple(filter_subtypes(p) for p in production.rhs())
            return Production(l, r)

        def create_terminals(tagged_text):
            """
            We need to add terminals to our grammar as well. This takes a tagged
            sentence, e.g. [('The', 'DT'), ...] and returns a list of terminal
            productions.
            The tagged text may also be a list of tagged sentences, in which case
            we add all words from these sentences to terminal productions
            """
            if isinstance(tagged_text[0], tuple):
                return set(Production(Nonterminal(tag), (word,))
                           for word, tag in tagged_text)
            else:
                return set(Production(Nonterminal(tag), (word,))
                           for tagged_sentence in tagged_text
                           for word, tag in tagged_sentence)

        def get_trees(tagged_sentence):
            """
            Returns all possible parse trees from a single sentence (presumably in the corpus).
            The sentence is POS-tagged, and this returns a list of Trees.
            """
            parsing = self.parser.parse(word for word, pos in tagged_sentence)
            return [tree for tree in parsing]

        def n_productions(parse_tree, production):
            """Returns the number of productions of type `production` in the parse_tree"""
            productions = list(parse_tree.subtrees(filter=lambda t: t.label() == production))
            return len(productions) 

        self.corpus = corpus
        self.productions = None
        self.branch_factor = None  # the average length of the rhs
        self.grammar = None
        self.parser = None

        treebank = nltk.corpus.treebank.parsed_sents()
        tb_productions = [production
                          for tree in treebank
                          for production in tree.productions()
                          if isinstance(production.rhs()[0], Nonterminal)]

        # simplify the productions
        simplified_productions = [filter_prod(p) for p in tb_productions]
        counter = Counter(simplified_productions)
        top_prods = {p for p, c in counter.most_common(250)}  # magic number
        self.productions = top_prods
        branch_factor = sum(len(p.rhs())
                            for p in self.productions) / len(self.productions)

        self.productions |= create_terminals(self.corpus)
        self.grammar = CFG(start=Nonterminal(
            'S'), productions=self.productions)
        self.parser = nltk.ChartParser(self.grammar)
        all_parse_trees = (get_trees(sentence) for sentence in corpus)

        # create statistics sentence by sentence
        self.stats = []
        for i, sentence_parse_trees in enumerate(all_parse_trees):
            print(i)
            sent_stats = {}
            if sentence_parse_trees:
                n_trees = len(sentence_parse_trees)
                avg_tree_depth = sum(tree.height()
                                     for tree in sentence_parse_trees) / n_trees
                max_tree_depth = max(tree.height()
                                     for tree in sentence_parse_trees)
                avg_noun_phrases = sum(n_productions(tree, 'NP')
                                       for tree in sentence_parse_trees) / n_trees
                avg_prep_phrases = sum(n_productions(tree, 'PP')
                                       for tree in sentence_parse_trees) / n_trees
                avg_sbars = sum(n_productions(tree, 'SBAR')
                                       for tree in sentence_parse_trees) / n_trees
                avg_nonterminals = sum(len(tree.productions())
                                       for tree in sentence_parse_trees) / n_trees
            else:
                n_trees = 1  # presumably there's at least one.
                avg_tree_depth = log(len(corpus[i])/log(branch_factor)) + 5
                max_tree_depth = math.ceil(avg_tree_depth)

            chart = self.parser.chart_parse((word for word, pos in corpus[i]))
            possible_parses = len(chart.edges())

            sent_stats['n_trees'] = n_trees
            sent_stats['max_tree_depth'] = max_tree_depth
            sent_stats['avg_tree_depth'] = avg_tree_depth
            sent_stats['possible_parses'] = possible_parses
            sent_stats['avg_noun_phrases'] = avg_noun_phrases
            sent_stats['avg_prepositional_phrases'] = avg_noun_phrases
            sent_stats['avg_sbars'] = avg_sbars
            sent_stats['avg_nonterminals'] = avg_nonterminals
            # etc.
            print(sent_stats)
            self.stats.append(sent_stats)

    def get_stats(self):
        """
        Combines all the statistics together
        """
        a_t = sum(s['n_trees'] for s in self.stats) / len(self.stats)
        m_t_d = max(s['max_tree_depth'] for s in self.stats)
        a_t_d = sum(s['avg_tree_depth'] for s in self.stats) / len(self.stats)
        a_p_p = sum(s['possible_parses'] for s in self.stats) / len(self.stats)
        a_n_p = sum(s['avg_noun_phrases'] for s in self.stats) / len(self.stats)
        a_p_p = sum(s['avg_prepositional_phrases'] for s in self.stats) / len(self.stats)
        a_s = sum(s['avg_sbars'] for s in self.stats) / len(self.stats)
        a_n = sum(s['avg_nonterminals'] for s in self.stats) / len(self.stats)

        return {
            'avg_trees': a_t,
            'max_tree_depth': m_t_d,
            'avg_tree_depth': a_t_d,
            'avg_possible_parses': a_p_p,
            'avg_noun_phrases': a_n_p,
            'avg_prepositional_phrases': a_p_p,
            'avg_sbars': a_s,
            'avg_nonterminals': a_n,
        }

In [ ]:
# Test stats class
tagged_sentences = tag_sentences('The quick brown fox jumps over the lazy dog. ' +
                                 'Now is the time for all good men to come to the aid of their party.')

tree_stats = TreeStats(tagged_sentences)
tree_stats.get_stats()

In [ ]:
# Test stats class
tagged = tag_corpus(c5)
tree_stats = TreeStats(tagged[1:6])
tree_stats.get_stats()

In [ ]:
# Test globals
productions = None
init_productions()
print(sum(len(p.rhs()) for p in productions), 'in', len(productions))
print(branch_factor)

In [ ]:
# Test all of these:
tagged_sentences = tag_sentences('The quick brown fox jumps over the lazy dog. ' +
                                 'Now is the time for all good men to come to the aid of their party.')
print(create_terminals(tagged_sentences))

t = get_trees(tagged_sentences[0])
print('n_trees:', n_trees(t))
print('max_tree_depth:', max_tree_depth(t))
print('avg_tree_depth:', avg_tree_depth(t))
print('*' * 80)
t = get_trees(tagged_sentences[1])
print('n_trees:', n_trees(t))
print('max_tree_depth:', max_tree_depth(t))
print('avg_tree_depth:', avg_tree_depth(t, tagged_sentences[1]))
print('*' * 80)
print('combined:', combined(tagged_sentences[0]))

## Parse-tree-based Metrics

These use the parse trees to measure things. We'll start with simple measurements, and build on top of those.

In [ ]:
def n_productions(parse_tree, production):
    """Returns the number of productions of type `production` in the parse_tree"""
    productions = list(parse_tree.subtrees(filter=lambda t: t.label() == production))
    return len(productions)
    
def n_noun_phrases(parse_tree):
    """Return the number of noun phrases in the parse tree"""
    return n_productions(parse_tree, 'NP')

def n_prepositional_phrases(parse_tree):
    """Returns the number of prepositional phrases in the parse tree"""
    return n_productions(parse_tree, 'PP')

def n_nonterminals(parse_tree):
    """Find the number of nonterminal productions in a parse tree"""
    return len(parse_tree.productions())

def n_sbars(parse_tree):
    """Find the number of SBAR's in a parse tree"""
    init_productions()
    return n_productions(parse_tree, 'SBAR')

def n_negations(sentence):
    """
    Returns the number of negations in the sentence
    """
    init_productions()
    return 0  # TODO -- how!?

In [ ]:
# Test all of these:
tagged_sentences = tag_sentences('The quick brown fox jumps over the lazy dog. ' +
                                 'Now is the time for all good men to come to the aid of their party.')
fox_trees = get_trees(tagged_sentence)

print('n_productions:', n_productions(fox_trees[8], 'NP'))  # should be 9
print('n_noun_phrases:', n_noun_phrases(fox_trees[8]))  # should be 9
print('n_prepositional_phrases:', n_prepositional_phrases(fox_trees[8]))  # should be 1
print('n_nonterminals:', n_nonterminals(fox_trees[8]))  # should be 1
print('n_sbars:', n_sbars(fox_trees[800]))  # should be 1



# print('max_tree_depth:', max_tree_depth(tagged_sentences[0]))
# print('avg_tree_depth:', avg_tree_depth(tagged_sentences[0]))
# print('*' * 80)
# print('n_trees:', n_trees(tagged_sentences[1]))
# print('max_tree_depth:', max_tree_depth(tagged_sentences[1]))
# print('avg_tree_depth:', avg_tree_depth(tagged_sentences[1]))

In [ ]:
f = fox_trees[800]
print(len(f.productions()))
f

## Readability

In this part I:

1. Calculate all of these stats for the pieces of the corpus

2. Calculate the SMOG scores

3. Figure out some relationship $f$ as above.`

In [ ]:
# TODO:
# 1. parse an entire corpus at once. √
# 2. get a parser for it, relatively optimized
# 3. get the stats for it

def tag_corpus(corpus):
    """
    The corpus is composed of newline-separated paragraphs of sentences.
    `tag_corpus` converts this into a list of POS-tagged sentences,
    ignoring paragraphs.
    """
    return tag_sentences(corpus)

def combined_textual_stats(corpus):
    """
    For a corpus (a list of POS-tagged sentences), determine all statistics as above
    and return a dictionary of these.
    """
    n_s = len(corpus)
    n_w = sum((n_words(sentence) for sentence in corpus))
    a_w_l = sum(sum(word_lengths(sentence)) for sentence in corpus) / n_w
    s = sum( sum(syllables(sentence)) for sentence in corpus)
    n_m_w = sum((n_monosyllable_words(sentence) for sentence in corpus))
    n_p_w = sum((n_polysyllable_words(sentence) for sentence in corpus))
    
    pos = {pos for sentence in corpus for pos in POSs(sentence)}
    n_pos = len(pos)
    n_pro = sum(n_pronouns(sentence) for sentence in corpus)
    n_r_p = sum(n_repeated_possessives(sentence) for sentence in corpus)
    n_r_a = sum(n_repeated_adverbs(sentence) for sentence in corpus)

    return {
            'n_sents': n_s,
            'n_words': n_w,
            'avg_word_length': a_w_l,
            'syllables': s,
            'n_monosyllable_words': n_m_w,
            'n_polysyllable_words': n_p_w,
            'n_POSs': n_pos,
            'n_pronouns': n_pro,
            'n_repeated_possessives': n_r_p,
            'n_repeated_adverbs': n_r_a
           }

def combined_tree_stats(corpus):
    """
    For a corpus (a list of POS-tagged sentences), determine all statistics as above
    and return a dictionary of these.
    """
    n_s = len(corpus)

    p_p = sum(possible_parses(sentence) for sentence in corpus)
    n_t = sum(n_trees(sentence) for sentence in corpus)
    m_t_d = max(max_tree_depth(sentence) for sentence in corpus)
    a_t_d = sum(avg_tree_depth(sentence) for sentence in corpus) / n_s  # This isn't quite accurate, but it's okay
    
    return {
        'possible_parses': p_p,
        'n_trees': n_t,
        'max_tree_depth': m_t_d,
        'avg_tree_depth': a_t_d
    }

In [ ]:
max_tree_depth(tagged[0])
# I need to: 1. Not keep recreating a parser
# 2. Reuse the trees I create already

In [ ]:
file = open('Corpora/third-grade.txt')
c3 = file.read()
file.close()
file = open('Corpora/fourth-grade.txt')
c4 = file.read()
file.close()
file = open('Corpora/fifth-grade.txt')
c5 = file.read()
file.close()

tagged = tag_corpus(c3)
print(flesch_kincaid_grade_level(tagged))
tagged = tag_corpus(c4)
print(flesch_kincaid_grade_level(tagged))
tagged = tag_corpus(c5)
print(flesch_kincaid_grade_level(tagged))

print(combined_textual_stats(tagged))

print(combined_tree_stats(tagged))

In [ ]:
tokenized = (t for s in c5.split('\n') for t in nltk.sent_tokenize(s))
tagged = tag_corpus(c5)
list(zip([n_repeated_adverbs(sentence) for sentence in tagged], tokenized))